In [2]:
import requests
import json
import re
import os
import pandas as pd
import logging
import time


def Prompt_Gen(case):
    #addtion = ""
    return f"""## Instruction:

### Task Description:
You are assigned to determine and interpret the presence and meaning of a specific licensing term within a portion of an open-source software license. Instead of reviewing the entire license, you will be provided with only those sections that are directly relevant to the specified term, along with some contextual license information. Your goal is to analyze these excerpts and identify whether they address the term and, if so, the term’s implications for users according to predefined categories.

To classify the term’s meaning, you may only use the following Simplified Markers: CAN, CANNOT, MUST, MUST NOT, OPTIONAL, UNCLEAR, and NOT SPECIFIED. Your analysis should strictly adhere to this set of markers without any additional terminology to ensure consistency. If the meaning is ambiguous, or if the term is absent, you must still choose one of the above markers and provide a supporting explanation using the provided license content only.

### Simplified Markers and Their Usage:
- **CAN**: Use this marker if the license explicitly permits the action associated with the term (e.g., modifying, distributing).
- **CANNOT**: Use this marker if the license explicitly restricts or prohibits the action (e.g., hold liable).
- **MUST**: Use this marker if the license mandates the action or condition (e.g., requiring that copyright notices be retained).
- **MUST NOT**: Use this marker if the license mandates that the action must not be done (e.g., not using a trademark without permission).
- **OPTIONAL**: Use this marker if the license mentions the action or term as optional or suggested, but not required (e.g., include license, include copyright).
- **UNCLEAR**: Use this marker if the term's meaning is ambiguous based on the license content or if the condition cannot be clearly determined.
- **NOT SPECIFIED**: Use this marker if the license does not mention the term or its related actions directly.


## Input:
The following structured information will be provided to help determine the status of the term in the license:

- License Information:
  - License Name: {case['license_name']}.
  - License Category: {case['license_info']['category']}.

- Specific Term to Evaluate:
  - Term Name: {case['license_terms']}.
  - Term Description: {case['terms_description']}.

- License Content for Analysis:
  - Relevant Lines: {case['content_lines']}.
  - Content Excerpts: {case['license_content']}.


## Output:

### Expected Output Format:
Generate a JSON object with the following keys:
- term: The name of the evaluated term.
- marker: One of the following values: "CAN", "CANNOT", "MUST", "MUST NOT", "OPTIONAL", "UNCLEAR", or "NOT SPECIFIED".
- explanation: A brief statement containing the line number and a relevant phrase or keyword from the license content that supports the marker selection. Try to use the original license text for explanations, the shorter the better!

### Example Output:
```json
{{
  "term": "Modify",
  "marker": "CAN",
  "explanation": "12: 'may modify and distribute derivative works.'"
}}
```

### Notes:
- Ensure that the marker strictly adheres to one of the specified values.
- The explanation should focus on providing a clear, concise justification for the marker using specific references from the license content.

"""


def Modeling(prompt):
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": "qwen2:72b",
            "prompt": f"{prompt}",
            "stream": False
        }
    ).json()
    full_response = response['response']
    return full_response

def process_a_case(license_case):
    print(f"Processing license: {license_case['license_name']}. Term: {license_case['license_terms']}     $$$ PLEASE WAITING $$$")
    prompt = Prompt_Gen(license_case)
    #print(prompt)
    
    if (len(prompt.encode('utf-8'))) > 128000:
        return
    
    if license_case['license_content'] == '':
        response = 'None'
    else:
        # 大模型判断
        #response = ''
        response = Modeling(prompt)

    result_dict = {
        "license_name": license_case['license_name'],
        'license_terms': license_case['license_terms'],
        "mistral_large_response": response,
    }
    
    json_data = json.dumps(result_dict, ensure_ascii=False)   
    
    print(json_data)
    
    with open(log_path, 'a+', encoding='utf-8') as f:
        f.write(json_data + '\n')


def is_license_processed(license_name, license_terms, log_path):
    """检查指定许可证是否已经在日志中处理过"""
    if not os.path.exists(log_path):
        return False
    
    with open(log_path, 'r', encoding="utf-8") as log_file:
        current_json = ""
        inside_json = False

        # 按行读取文件，逐行解析 JSON 对象
        for line in log_file:
            line = line.strip()
            
            # 检测是否是 JSON 对象的开头, 尝试解析单行 JSON
            if line.startswith("{") and line.endswith("}"):
                try:
                    json_data = json.loads(line)
                    if json_data.get("license_name") == license_name and json_data.get("license_terms") == license_terms:
                        return True
                except json.JSONDecodeError as e:
                    print(f"Failed to decode JSON: {e}")
                    
                    # # 如果解析失败，进入多行模式
                    # inside_json = True
                    # current_json = line
            
            # 多行拼接 JSON 对象
            # elif inside_json:
            #     current_json += line
            #     if line.endswith("}"):
            #         try:
            #             # 尝试解析完整的多行 JSON
            #             json_data = json.loads(current_json)
            #             if json_data.get("license_name") == license_name and json_data.get("license_terms") == license_terms:
            #                 return True
            #             inside_json = False  # 重置状态
            #         except json.JSONDecodeError:
            #             # 继续拼接，直到完成 JSON
            #             continue
    
    return False


if __name__ == '__main__':    
    log_path = "/home/keqiang/Benchmark/license_data/license_llm/term/logging/2024-10-29-qianwen.log"
    license_terms_spdx = '/home/keqiang/Benchmark/license_data/license_manual/license_terms_spdx.json'
    license_info_spdx = '/home/keqiang/Benchmark/license_data/license_manual/has_spdx_spdx.json'
    terms = '/home/keqiang/Benchmark/license_data/license_info/terms.csv'
    
    with open(license_terms_spdx, mode='r', encoding="utf-8") as f:
        license_terms_data = json.load(f)
        
    with open(license_info_spdx, mode='r', encoding="utf-8") as f:
        license_info_data = json.load(f)

    df = pd.read_csv(terms)
    
    #flag = 2
    
    for license_items in license_terms_data:
        license_name = license_items['license_name']
        terms = license_items['terms']
        
        for license_terms, details in terms.items():
            content_lines = details["lines"]
            license_content = details["content"]
            
            terms_description = df.loc[df['Term'] == license_terms, 'Description'].values
            
            license_summary = next((item["license_summary"] for item in license_info_data 
                    if item["license_summary"]["license_information"]["filename"] == license_name), None)
            license_info = license_summary['license_information']

            # 检查日志文件中是否已处理该许可证
            if is_license_processed(license_name, license_terms, log_path):
                print(f"Skipping {license_name}, {license_terms}, already processed.")
                continue

            # 将扫描结果和license信息整合到字典中
            license_case = {
                'license_info': license_info,               # 许可证的文件信息
                'license_name':license_name,                # 许可证名称，含有.license
                'license_terms': license_terms,             # 需要判定的具体条款
                'content_lines': content_lines,             # 条款在许可证文本中的定位
                'license_content': license_content,         # 许可证文本中与条款相关的语句
                'terms_description': terms_description      # 条款的具体解释
            }
            
            process_a_case(license_case)



Processing license: bsd-zero.license. Term: Distribute     $$$ PLEASE WAITING $$$


KeyboardInterrupt: 